In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle as pkl
import glob
import numpy as np
import cv2
from img_load import load_bimg
from NPSmethods import *

In [2]:
psat=277

imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

choices = {
    "if_Save"                     : False,
    "do_Fit"                      : True,
    "plot_2dGas"                  : False,
    "plot_NoisePowSpec"           : False,
    "plot_NoisePowSpec_LineCut"   : False,
    "plot_Pupil"                  : False,
    "plot_PSF"                    : False,
    "plot_PSF_LineCut"            : False,
    "plot_PSF_abs2"               : False,
    "plot_PSF_abs2_LineCut"       : False,
    "plot_Sk"                     : False,
    "normalize"                   : False
}

path = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image'

resDir = r"C:\Users\eric0\Jupyter Notebooks\Cold Physics\results"

trapRegion = (slice(100, 200), slice(100, 200))
    # the region where the atoms located, [(xmin, xmax), (ymin, ymax)] (pixel)

noiseRegion = (slice(0, 300), slice(0, 300))
    # the region chosen for noise analysis, [(xmin, xmax), (ymin, ymax)] (pixel)

numOfImgsInEachRun = 50      # number of images for each run of analysis

rampingParameter = 0

rawimages = []
parameters = []

for folder in glob.iglob(path + "\\*"):
    result=load_bimg(folder + "\\", raw=True)
    odimg=-np.log(result['raw1']/result['raw2'])-(result['raw1']-result['raw2'])/psat
    _, _, _, _, popt, _ = doCalibration(folder, resDir, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
    for i in range(len(odimg)):
        rawimages.append(odimg[i])
        parameters.append(popt)


load data from C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image\211412\
od size (0,)
tau     =  0.5000 : describe the decaying of transmission efficiency with radius
S0      = -1.3693 : spherical aberration
alpha   =  0.0720 : astigmatism
beta    =  0.1926 : defocus
delta_s =  1.4378 : phase imprint by atom scattering

load data from C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image\221529\
od size (0,)
tau     =  1.0526 : describe the decaying of transmission efficiency with radius
S0      = -1.0497 : spherical aberration
alpha   =  1.0266 : astigmatism
beta    =  0.8316 : defocus
delta_s = -0.0667 : phase imprint by atom scattering



In [22]:
scaler = StandardScaler()
yscaler = scaler.fit(np.array(parameters))
scaledpara = scaler.transform(np.array(parameters))
rawimages = np.array(rawimages).reshape((147,-1))
trainin, testin, trainout, testout = train_test_split(rawimages, scaledpara, test_size=0.25, shuffle=True)

In [23]:
pkl.dump(trainin, open(resDir + "\\trainin.pkl", "wb"))
pkl.dump(trainout, open(resDir + "\\trainout.pkl", "wb"))
pkl.dump(testin, open(resDir + "\\testin.pkl", "wb"))
pkl.dump(testout, open(resDir + "\\testout.pkl", "wb"))
pkl.dump(yscaler, open(resDir + "\\yscaler.pkl", "wb"))

In [24]:
print(np.shape(trainin))

(110, 89700)


In [25]:
print(scaler.mean_)

[ 0.47161555  0.57893775 -1.3236886   0.20832933 -1.66760626  0.28390575
  1.22289219]
